In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Node(ABC):
    """Base class for all nodes in the abstract syntax tree (AST) of a regex pattern."""

    @abstractmethod
    def matches(self, string: str) -> bool:
        """Returns a boolean if the node matches the given string."""
        raise NotImplementedError("Node is an abstract class and cannot be instantiated.")

class Literal(Node):
    """A literal character in a regex pattern."""
    def __init__(self, char):
        self.char = char

    def matches(self, string: str) -> bool:
        return string and len(string) > 0 and string[0] == self.char

    def __repr__(self):
        return f"Literal({self.char!r})"
    
class CharacterClass(Node):
    """A character class in a regex pattern."""
    def __init__(self, chars):
        self.chars = chars

    def matches(self, string: str) -> bool:
        return string and len(string) > 0 and string[0] in self.chars

    def __repr__(self):
        return f"CharacterClass({self.chars!r})"

class Concat(Node):
    """A concatenation of two nodes in a regex pattern."""
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def matches(self, string: str) -> bool:
        # starting with the left node, check if the string matches the left node
        # if it does, then check if the string matches the right node
        # if it does, then return True
        for split in range(len(string) + 1):
            if self.left.matches(string[:split]) and self.right.matches(string[split:]): return True
        return False

    def __repr__(self):
        return f"Concat({self.left!r}, {self.right!r})"
    
class Repitition(Node):
    """A repitition of a node in a regex pattern."""
    def __init__(self, child: Node):
        self.child = child

    def matches(self, string: str) -> bool:
        if self.child.matches(''): return True # match 0 occurences

        # match one or more occurences
        i = 0
        while i < len(string):
            if not self.child.matches(string[i]):
                break
            i += 1

        # if we matched something and the rest of the string matches, return True
        return i > 0 and self.matches(string[i:])

def parse_regex(pattern):
    """Parses a regex pattern string and constructs an AST."""
    if not pattern: raise ValueError("Pattern cannot be empty.")

    # convert the pattern string into a list of tokens
    tokens = [Literal(char) for char in pattern]

    # reduce the list of tokens into a single AST
    # use left associative concatentation to reduce the list of tokens
    ast = tokens[0]
    for token in tokens[1:]:
        ast = Concat(ast, token)
    return ast

def match(ast, string: str):
    """Matches a string against the AST of a regex pattern."""

    def _match_eof(string: str): return string == '' # helper function to match end of string

    if isinstance(ast, Literal): # if the node is a literal, check if the string matches the first character
        return ast.matches(string) and _match_eof(string[1:])
    elif isinstance(ast, CharacterClass): # if the node is a character class, check if the string matches the first character
        return ast.matches(string) and _match_eof(string[1:])
    elif isinstance(ast, Concat): # if the node is a concatenation, iteratively check if the string matches the left node and the right node
        return any(match(ast.left, string[:split]) and match(ast.right, string[split:]) for split in range(len(string) + 1)) # left associative
    elif isinstance(ast, Repitition): # if the node is a repitition, iteratively check if the string matches the child node
        return (match(ast.child, string) and match(ast, string[1:])) or _match_eof(string) # match 0 or more occurences
    # TODO add more node types here...
    else:
        raise ValueError(f"Unknown node type: {type(ast)}")